In [1]:
import numpy as np
import torch
import torch.nn as nn

from flash_ansr import FlashANSRDataset, get_path
from flash_ansr.models.encoders.pre_encoder import PreEncoder

In [2]:
# dataset = FlashANSRDataset.from_config(get_path('configs', 'v6.0', 'dataset_val.yaml'))
dataset = FlashANSRDataset.from_config(get_path('data', 'ansr-data', 'test_set', 'feynman', 'dataset.yaml'))

pre_encoder = PreEncoder(input_size=4, mode='ieee-754')

Compiling Skeletons: 100%|██████████| 42/42 [00:00<00:00, 33631.30it/s]


In [ ]:
for single_element_batch in dataset.iterate(size=100000, n_support=16, verbose=True):
    input_ids, x_tensor, y_tensor, labels, constants = FlashANSRDataset.collate_batch(single_element_batch, device='cpu')

    x_tensor = x_tensor.unsqueeze(0)
    y_tensor = y_tensor.unsqueeze(0)

    if not np.isfinite(y_tensor.std().numpy()):
        print(f"{y_tensor.std()=}")

    r = torch.randn_like(y_tensor)
    y_tensor_noisy = y_tensor + (y_tensor.std() * r)

    if not np.all(np.isfinite(y_tensor_noisy.numpy())):
        print(f"{y_tensor_noisy.std()=}")
        print(f"{y_tensor.std()=}")
        print(f"{r.min()=}")
        print(f"{r.max()=}")

    # Pad the x_tensor with zeros to match the expected maximum input dimension of the set transformer
    pad_length = 4 - x_tensor.shape[-1] - y_tensor_noisy.shape[-1]
    if pad_length > 0:
        x_tensor = nn.functional.pad(x_tensor, (0, pad_length, 0, 0, 0, 0), value=0)

    data_tensor = torch.cat([x_tensor, y_tensor_noisy], dim=-1)

    data_tensor_pre_encoded = pre_encoder(data_tensor)

    if not np.all(np.isfinite(data_tensor_pre_encoded.numpy())):
        print(f"{data_tensor_pre_encoded.std()=}")

Generating data:  88%|████████▊ | 88475/100000 [01:59<00:15, 739.14it/s, reject_rate=0.26%]


y_tensor_noisy.std()=tensor(nan)
y_tensor.std()=tensor(7.8490e+37)
r.min()=tensor(-1.6074)
r.max()=tensor(2.7395)


RuntimeError: shape mismatch: value tensor of shape [16, 4, 16] cannot be broadcast to indexing result of shape [1, 16]